In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, 
    AdaBoostClassifier, StackingClassifier
)
from sklearn.model_selection import (
    cross_val_score, StratifiedKFold, train_test_split, cross_val_predict, 
    GridSearchCV, RandomizedSearchCV
)
from sklearn.metrics import (
    accuracy_score, f1_score, make_scorer, roc_auc_score, 
    confusion_matrix, classification_report
)
from sklearn.preprocessing import (
    StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import RFECV

from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import make_pipeline as make_imblearn_pipeline

from feature_engine.encoding import CountFrequencyEncoder
from xgboost import XGBClassifier

from scipy.stats import randint, uniform

import random

from Functions_Classes import *

from sklearn.base import BaseEstimator, TransformerMixin

pd.set_option('display.max_columns', None)
np.random.seed(0)


In [2]:
df = pd.read_excel("C:/Users/Cagan Deliktas/Desktop/ProjectDataMining2/DM2_DataCraft/data/training_data.xls")
X_test = pd.read_excel("C:/Users/Cagan Deliktas/Desktop/ProjectDataMining2/DM2_DataCraft/data/test_data_no_target.xls")

df = df.loc[:, df.columns != 'Perform']
#df = df.loc[:, df.columns != 'Group']


X_train = df.loc[:, df.columns != 'Class']
y_train = df['Class']

In [3]:
numeric_columns = X_train.loc[:, ~X_train.columns.isin(['Group'])].columns.to_list()
X_train.loc[:, numeric_columns] = X_train.loc[:, numeric_columns].replace(
    {
        'NA': np.nan, 
        '': np.nan,
        ' ': np.nan
    }
).astype(float)

In [4]:
numeric_columns = X_test.loc[:, ~X_test.columns.isin(['Group'])].columns.to_list()
X_test[numeric_columns] = X_test.loc[:, numeric_columns].replace(
    {
        'NA': np.nan, 
        '': np.nan,
        ' ': np.nan
    }
).astype(float)

In [5]:
X_train.head()

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,dI1,dI2,dI3,dI4,dI5,dI6,dI7,dI8,dI9,dI10,dI11,dI12,dI13,dI14,dI15,dI16,dI17,dI18,dI19,dI20,dI21,dI22,dI23,dI24,dI25,dI26,dI27,dI28,dI29,dI30,dI31,dI32,dI33,dI34,dI35,dI36,dI37,dI38,dI39,dI40,dI41,dI42,dI43,dI44,dI45,dI46,dI47,dI48,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58
0,G9,0.136495,-0.028429,-0.037772,-0.232459,-0.016222,-0.187506,-0.322545,-0.043743,0.125389,-0.014757,-0.033105,0.303035,-0.093811,-0.598917,-0.271292,-0.256749,-0.100146,-0.045525,-0.078422,-0.060129,-0.069528,-0.052432,-0.114432,-0.104989,0.342845,-0.159417,0.006772,-0.303193,-0.163287,-0.080599,-0.828880,-1.064215,-0.547067,-0.540497,-0.676045,-0.305007,-0.507724,-0.191437,-0.087362,-0.856151,0.802525,0.733080,0.006512,0.533290,0.195197,0.058094,-0.228889,-0.150821,-0.104986,-0.026743,0.188312,-0.250701,-0.101190,-0.357521,-0.527956,0.611385,-0.092714,-0.055733,-0.065709,-0.002144,-0.004367,-0.079805,0.178280,0.078155,0.072802,0.002090,0.211770,-0.003073,-0.188447,0.117769,0.001613,-0.024223,0.103204,0.032484,0.002688,0.000765,-0.004447,0.148967,-0.018521,-0.014110,-0.001996,-0.002369,-0.120036,0.013172,-0.215571,-0.021999,0.001728,-0.000050,-0.012120,-0.040172,-0.060103,-0.059464,-0.044899,0.015735,0.022919,-0.003106,0.001233,-0.002339,0.040628,0.411684,0.073090,0.526222,0.071060,-0.019531,0.359889,-0.020476,0.057151,0.077110,0.102563,0.188481,-0.016027,-0.135451,-0.189667,0.250967,0.022171,-0.004265
1,G5,-0.714522,-0.042137,-0.052968,-0.796862,-0.018394,0.070102,-0.076321,-0.063864,-1.045521,-0.037353,-0.792515,-1.082483,0.025798,-0.833652,-0.625088,-0.333608,0.072579,-0.046963,0.223022,-0.605902,-0.131099,-0.235929,-0.073920,-0.063247,-0.798768,-0.899983,1.388771,-0.248677,-0.058083,-0.014470,0.092095,0.561368,0.224819,0.223190,0.098852,-0.128227,-0.215876,-0.007164,-0.035260,-0.123911,-0.089751,-0.094963,0.362818,0.011107,-1.506356,-0.573679,-0.955222,-0.818880,-1.063295,-1.022679,-1.336188,-0.612039,-0.061357,-0.482805,-0.017077,1.192135,-0.114981,-0.028074,-0.004451,-0.000536,-0.002288,-0.045597,-0.080639,-0.081924,-0.033862,-0.005111,-0.261836,0.000122,-0.045046,0.999854,-0.008835,-0.122379,-0.199892,0.013615,0.014404,-0.000405,0.021573,-0.024160,-0.037420,-0.012610,0.003007,0.003617,-0.106893,-0.394834,-0.132496,-0.027354,-0.129804,-0.066157,-0.494334,0.123781,0.284328,0.281308,0.212767,0.192042,0.146926,-0.118826,-0.039203,-0.256107,0.176622,0.168840,0.487752,0.029464,0.014232,0.039633,0.025667,0.006626,0.005180,0.006128,-0.016375,0.020727,-0.006525,-0.018790,-0.098543,0.317744,-0.180502,-0.009215
2,G10,0.104791,-0.038188,-0.053191,0.620233,0.148587,0.489875,0.319274,-0.060246,0.053174,-0.025008,-0.456840,1.284450,-0.133470,3.207672,2.373230,1.304427,NaN,NaN,-0.361293,2.995661,NaN,-0.188988,-0.044158,-0.024550,-0.586562,-0.176292,-1.013037,0.066912,0.219649,0.154490,2.370951,1.384675,0.489152,0.484715,0.367301,0.749572,0.669410,0.423228,0.226897,3.227283,-0.329997,-0.327579,-1.033898,0.014531,0.211889,-1.197156,2.860444,NaN,3.584223,NaN,1.272375,7.427558,-0.182816,-2.713205,-1.877595,-0.568691,0.224945,0.052749,0.377640,0.002656,0.001226,0.226060,0.207653,0.270327,0.283061,0.002934,0.454366,0.004264,0.188623,-0.265918,0.000000,2.063796,1.076458,0.240011,NaN,NaN,-0.028327,1.764826,NaN,0.005847,-0.011166,-0.012626,-0.010822,0.056514,-0.100007,-0.216081,-0.127274,-0.056206,0.175751,-0.011770,0.493157,0.487919,0.438576,0.574623,0.564379,-0.165933,-0.051256,0.410379,0.056624,0.047592,0.000000,-0.020586,0.237539,0.017314,0.516667,NaN,0.404158,NaN,0.272937,0.774169,-0.007144,0.123954,0.000000,-0.110103,0.186669,-0.030720
3,G2,-0.532847,-0.006582,-0.023377,1.306702,-0.068909,0.048024,-0.119481,-0.021057,-1.012916,-0.011783,1.206727,0.311773,-0.005928,3.869459,-1.064793,0.107702,-0.126984,-0.044360,-0.181023,-0.691971,NaN

## Shape

In [6]:
X_train.shape

(8000, 117)

# Detect Categorical and Numerical Columns

In [7]:
categorical_features = ['Group']
numerical_features = [col for col in X_train.columns if col not in categorical_features + ['Class']]

## Preprocessing Pipeline

In [8]:
numerical_pipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)), 
    ('scaler', RobustScaler()),
])

categorical_pipeline = Pipeline([
    ('one_hot', OneHotEncoder())
])

preprocessor = ColumnTransformer([
    ('num', numerical_pipeline, numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

#rfecv_classifier = DecisionTreeClassifier(random_state=42)
#selector = RFECV(estimator=rfecv_classifier, step=1, cv=StratifiedKFold(3), scoring=matrix_error_function)

# Classification Models

In [9]:
final_classifiers = {
    'random_forest': RandomForestClassifier(random_state=42),
    'xgboost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'gradient_boosting': GradientBoostingClassifier(random_state=42)
}

### 1st Pipeline

In [10]:
y_train_mapped = y_train.copy()
y_train_mapped[y_train == -1] = 0
y_train_mapped[y_train == 0] = 1
y_train_mapped[y_train == 1] = 2

cv_scores = {}

for name, classifier in final_classifiers.items():
    pipeline = make_imblearn_pipeline(
        preprocessor,
        SMOTE(random_state=42),
        #selector,
        classifier
    )
   
    scores = cross_val_score(pipeline, X_train, y_train_mapped, cv=StratifiedKFold(5), scoring=matrix_error_function, n_jobs=-1)
   
    cv_scores[name] = scores
    print(f'Cross-validation scores for {name}: {scores}')
    print(f'Average cost error for {name}: {np.mean(scores)}')

Cross-validation scores for random_forest: [-0.910625 -0.921875 -0.93375  -0.918125 -0.94375 ]
Average cost error for random_forest: -0.9256249999999999
Cross-validation scores for xgboost: [-0.92     -0.88     -0.895    -0.853125 -0.8875  ]
Average cost error for xgboost: -0.8871249999999999
Cross-validation scores for gradient_boosting: [-0.903125 -0.8775   -0.88     -0.880625 -0.89375 ]
Average cost error for gradient_boosting: -0.8869999999999999


### 2nd pipeline

In [12]:
y_train_mapped = y_train.copy()
y_train_mapped[y_train == -1] = 0
y_train_mapped[y_train == 0] = 1
y_train_mapped[y_train == 1] = 2

cv_scores = {}

for name, classifier in final_classifiers.items():
    pipeline = make_imblearn_pipeline(
        preprocessor,
        RandomUnderSampler(random_state=42),
        #selector,
        classifier
    )
   
    scores = cross_val_score(pipeline, X_train, y_train_mapped, cv=StratifiedKFold(5), scoring=matrix_error_function, n_jobs=-1)
   
    cv_scores[name] = scores
    print(f'Cross-validation scores for {name}: {scores}')
    print(f'Average cost error for {name}: {np.mean(scores)}')

Cross-validation scores for random_forest: [-0.906875 -0.894375 -0.916875 -0.895    -0.891875]
Average cost error for random_forest: -0.901
Cross-validation scores for xgboost: [-0.92125  -0.92     -0.876875 -0.881875 -0.87875 ]
Average cost error for xgboost: -0.8957499999999999
Cross-validation scores for gradient_boosting: [-0.91375  -0.873125 -0.885625 -0.879375 -0.91    ]
Average cost error for gradient_boosting: -0.892375


### 3rd Pipeline

In [14]:
y_train_mapped = y_train.copy()
y_train_mapped[y_train == -1] = 0
y_train_mapped[y_train == 0] = 1
y_train_mapped[y_train == 1] = 2

cv_scores = {}

for name, classifier in final_classifiers.items():
    pipeline = make_imblearn_pipeline(
        preprocessor,
        RandomOverSampler(random_state=42),
        #selector,
        classifier
    )
   
    scores = cross_val_score(pipeline, X_train, y_train_mapped, cv=StratifiedKFold(5), scoring=matrix_error_function, n_jobs=-1)
   
    cv_scores[name] = scores
    print(f'Cross-validation scores for {name}: {scores}')
    print(f'Average cost error for {name}: {np.mean(scores)}')

Cross-validation scores for random_forest: [-0.91     -0.903125 -0.949375 -0.933125 -0.91375 ]
Average cost error for random_forest: -0.921875
Cross-validation scores for xgboost: [-0.916875 -0.893125 -0.87875  -0.88125  -0.9075  ]
Average cost error for xgboost: -0.8955
Cross-validation scores for gradient_boosting: [-0.936875 -0.8675   -0.889375 -0.871875 -0.885   ]
Average cost error for gradient_boosting: -0.8901249999999999


### 4th Pipeline

In [15]:
y_train_mapped = y_train.copy()
y_train_mapped[y_train == -1] = 0
y_train_mapped[y_train == 0] = 1
y_train_mapped[y_train == 1] = 2

cv_scores = {}

for name, classifier in final_classifiers.items():
    pipeline = make_imblearn_pipeline(
        preprocessor,
        #RandomOverSampler(random_state=42),
        #selector,
        classifier
    )
   
    scores = cross_val_score(pipeline, X_train, y_train_mapped, cv=StratifiedKFold(5), scoring=matrix_error_function, n_jobs=-1)
   
    cv_scores[name] = scores
    print(f'Cross-validation scores for {name}: {scores}')
    print(f'Average cost error for {name}: {np.mean(scores)}')

Cross-validation scores for random_forest: [-0.90375  -0.8975   -0.9325   -0.874375 -0.9175  ]
Average cost error for random_forest: -0.905125
Cross-validation scores for xgboost: [-0.909375 -0.915625 -0.89375  -0.889375 -0.9075  ]
Average cost error for xgboost: -0.903125
Cross-validation scores for gradient_boosting: [-0.899375 -0.87875  -0.88125  -0.87375  -0.88875 ]
Average cost error for gradient_boosting: -0.884375


## Predictions Voting

In [75]:
file_path = "comb10_4.txt"
pd.DataFrame(yhat).to_csv(file_path, index=False, header=False)